# Smarter Betas - a regime-switching convex optimized model for factor investing
##### Created by: Izzat Aziz and Edward Chai (2021)
##### Modified from: Oprisor, Razvan, and Roy Kwon (2021)

### Theoretical framework
1. Hidden Markov Model (HMM): a statistical Markov model in which the system being modeled is assumed to be a Markov process (i.e. a sequence of possible events in which the probability of each event depends only on the state attained in the previous event) but with hidden states (i.e. unobservable).
2. Black Litterman Model (BL): A model that combines Markowitz's mean-variance optimization with Sharpe’s CAPM through a Bayesian approach (Black and Litterman, 1992).
3. Convex Optimization: Formulating investment decisions as a convex optimization problem that trades off expected return, risk and transaction costs (Boyd et al., 2017)

### Implementation
1. Fetch data from Bloomberg
2. For EACH asset, predict forward returns (i.e. T+1) by passing (T - rolling window : T) historical returns, volatility and macro factors into HMM. Predicted returns are calculated by taking the dot product of HMM's transition matrix and HMM's returns' mean given states matrix.
3. Generate BL posterior returns by taking HMM's returns as views (i.e. priors), market cap implied returns, and using ledoit-wolf's covariance shrinkage matrix (Ledoit and Wolf, 2001).
4. Generate max sharpe portfolio given posterior returns and ledoit-wolf'
4. (Part II) Identify the optimized asset allocation for a long-only portfolio by maximizing risk-adjusted returns given BL's posterior returns subject to costs and constraints

### PART II: https://github.com/nurizzataiman/Smarter-Betas/blob/main/Smarter%20Betas%20II%20(post-BQuant).ipynb
##### RUN SMARTER BETAS II (POST-BQUANT) USING THE SAVED EXCEL FILE AT THE END



In [1]:
# Install external libraries (This will take a while)
%install osqp 0.6.1
%install cvxpy 1.1.4 
%install PyPortfolioOpt 1.2.7
%install hmmlearn 0.2.5

Pypi package osqp-0.6.1 activated
Pypi package cvxpy-1.1.4 activated
Pypi package PyPortfolioOpt-1.2.7 activated
Pypi package hmmlearn-0.2.5 activated


### Once the external libraries are installed, run this cell to launch the app!

In [2]:
# Importing the basic libraries
import pandas as pd
import numpy as np
import bqplot.pyplot as plt
import matplotlib.pyplot as plt1
import bqviz as bqv
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

# Importing Hidden Markov, Black Litterman and Convex Portfolio libraries
from hmmlearn.hmm import GaussianHMM as ghmm
from pypfopt import black_litterman, risk_models, efficient_frontier

# Ignoring warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Importing Bloomberg Query library
import bql
bq = bql.Service()

import ipywidgets as widgets
import bqwidgets as bqw
import datetime as dt

start_date_label = widgets.Label(value="Start date (Friday)")
start_date1 = widgets.DatePicker(value=dt.datetime.today())
end_date_label = widgets.Label(value="End date (Friday)")
end_date1 = widgets.DatePicker(value=dt.datetime.today())
HBoxDates1 =widgets.HBox([start_date_label, start_date1])
HBoxDates2 =widgets.HBox([end_date_label, end_date1])

ticker_arr = []
ticker_label = widgets.Label('')
ticker_ac = bqw.TickerAutoComplete(value='M1WOMOM Index', yellow_keys=['index'])
ticker_button_add = widgets.Button(description='Add', button_style='info')
ticker_button_clear = widgets.Button(description='Clear', button_style='danger')

def add_ticker(btn):
    ticker_arr.append(ticker_ac.value)
    ticker_label.value = ticker_ac.value + " , " + ticker_label.value 

def remove_ticker(btn):
    ticker_arr.clear()
    ticker_label.value = ''

ticker_button_add.on_click(add_ticker)
ticker_button_clear.on_click(remove_ticker)

HBoxTickerAdd = widgets.HBox([ticker_ac,ticker_button_add])
HBoxTickerClear = widgets.HBox([ticker_button_clear,ticker_label])

macro_arr = []
macro_label = widgets.Label('')
macro_ac = bqw.TickerAutoComplete(value='USYC2Y10 Index', yellow_keys=['index'])
macro_button_add = widgets.Button(description='Add', button_style='info')

macro_button_clear = widgets.Button(description='Clear', button_style='danger')

def add_macro(btn):
    macro_arr.append(macro_ac.value)
    macro_label.value = macro_ac.value + " , " + macro_label.value

def remove_macro(btn):
    macro_arr.clear()
    macro_label.value = ''

macro_button_add.on_click(add_macro)
macro_button_clear.on_click(remove_macro)

HBoxMacroAdd = widgets.HBox([macro_ac,macro_button_add])
HBoxMacroClear = widgets.HBox([macro_button_clear,macro_label])

benchmark_ac = bqw.TickerAutoComplete(value='MXWO Index', yellow_keys=['index'])

Ticker_title = widgets.HTML(value='<b> Input Factor ETFs below </b>')
Macro_title = widgets.HTML(value='<b> Input Macros below </b>')
Benchmark_title = widgets.HTML(value='<b> Input Benchmark below </b>')

start_date_label.layout.width = "150px"
start_date1.layout.width='150px'
end_date_label.layout.width = "150px"
end_date1.layout.width='150px'
ticker_ac.layout.width = '400px'
ticker_button_add.layout.width='50px'
ticker_button_clear.layout.width='100px'
ticker_label.layout.width = '350px'
macro_ac.layout.width = '400px'
macro_button_add.layout.width='50px'
macro_button_clear.layout.width='100px'
macro_label.layout.width='350px'
benchmark_ac.layout.width ='450px'

BloombergButton = widgets.Button(description='Fetch WEEKLY data', button_style='success')
BloombergButton.layout.width = '450px'

BloombergInputBox = widgets.VBox([HBoxDates1, HBoxDates2,
                        Ticker_title, HBoxTickerAdd,HBoxTickerClear, 
                        Macro_title,HBoxMacroAdd,HBoxMacroClear, 
                        Benchmark_title, benchmark_ac,
                        BloombergButton])

df_price = pd.DataFrame() 
df_vol = pd.DataFrame()
df_mktcap = pd.DataFrame()
df_macro = pd.DataFrame()
df_benchmark = pd.DataFrame()
df_pct_ret = pd.DataFrame()
df_pct_vol = pd.DataFrame()

BloombergPlotBox = widgets.VBox([])

def get_Bloomberg(btn):

    # fetch user inputs
    tickers = ticker_arr
    macro_tickers = macro_arr
    benchmark = benchmark_ac.value
    start_date = start_date1.value
    end_date = end_date1.value

    # Creating a dataframe for ticker prices
    price = bq.func.rolling(bq.data.px_last(dates=bq.func.range('0d','0d'),fill='prev'),iterationdates=bq.func.range(start=start_date, end=end_date, frq='W'))
    price = bql.Request(tickers, {'Price': price})
    price = bq.execute(price)
    global df_price
    df_price = price[0].df()
    df_price = df_price.reset_index().pivot(index='DATE', columns='ID', values='Price')
    df_price.index.name = 'dates'

    # Creating a dataframe for ticker 10 day volatility
    vol = bq.func.rolling(bq.data.volatility(calc_interval='10d'),iterationdates=bq.func.range(start=start_date, end=end_date, frq='W' ))
    vol = bql.Request(tickers, {'Vol': vol})
    vol = bq.execute(vol)
    global df_vol
    df_vol = vol[0].df()
    df_vol = df_vol.reset_index().pivot(index='DATE', columns='ID', values='Vol')
    df_vol.index.name = 'dates'

    # Creating a dataframe for ticker market caps
    mcaps = bq.func.rolling(bq.data.cur_mkt_cap(dates=bq.func.range('0d','0d'),fill='prev'),iterationdates=bq.func.range(start=start_date, end=end_date, frq='W'))
    mcaps = bql.Request(tickers, {'Market cap': mcaps})
    mcaps = bq.execute(mcaps)
    global df_mktcap
    df_mktcap = mcaps[0].df()
    df_mktcap = df_mktcap.reset_index().pivot(index='DATE', columns='ID', values='Market cap').dropna()
    df_mktcap.index.name = 'dates'

    # Creating a dataframe for macro indicators
    macro = bq.func.rolling(bq.data.px_last(dates=bq.func.range('0d','0d'),fill='prev'),iterationdates=bq.func.range(start=start_date, end=end_date, frq='W'))
    macro = bql.Request(macro_tickers, {'Macro': macro})
    macro = bq.execute(macro)
    global df_macro
    df_macro = macro[0].df()
    df_macro = df_macro.reset_index().pivot(index='DATE', columns='ID', values='Macro')
    df_macro.index.name = 'dates'
    df_macro = df_macro.dropna()

    # Creating a dataframe for benchmark price
    bench_price = bq.func.rolling(bq.data.px_last(dates=bq.func.range('0d','0d'),fill='prev'),iterationdates=bq.func.range(start=start_date, end=end_date, frq='W'))
    bench_price = bql.Request(benchmark, {'Price': bench_price})
    bench_price = bq.execute(bench_price)
    global df_benchmark
    df_benchmark = bench_price[0].df()
    df_benchmark = df_benchmark.reset_index().pivot(index='DATE', columns='ID', values='Price')
    df_benchmark.index.name = 'dates'

    # Converting absolute returns and abslute volatility to percentage change period-on-period
    global df_pct_ret
    df_pct_ret = df_price.pct_change().dropna()
    global df_pct_vol
    df_pct_vol = df_vol.pct_change().dropna()

    for Q in range(0,len(df_price.columns)):
        df_price.iloc[:,Q] = pd.to_numeric(df_price.iloc[:,Q], downcast="float")

    PlotPrice = bqv.InteractiveLinePlot(df_price)
    BloombergPlotBox.children = [PlotPrice.show()]
    BloombergPlotBox.layout.width = '1000px'
    HideTemp1.children = [MainBoxTemp1]

BloombergButton.on_click(get_Bloomberg)

BloombergBox = widgets.HBox([BloombergInputBox,BloombergPlotBox])


# Modified from Falkner (2017):"Stock-Forecasting using HMM - A Model Averaging Approach" (https://github.com/Mojomotte/ML2-Stock-Forecasting-using-HMM---An-Model-Averaging-Approach)
# First implemented by Nguyen (2016): "Stock Price Prediction using Hidden Markov Model" (https://editorialexpress.com/cgi-bin/conference/download.cgi?db_name=SILC2016&paper_id=38)

# Creating an HMM training and predicting function to be used later on
def hmm_model(data, states, max_iterations=1000, conv=True, show=False):

    if show == True:
        print('Fitting and predicting... ', end='')

    # Creating a basic Hidden Markov Model
    model = ghmm(n_components = states,
                 covariance_type='full',
                 n_iter = max_iterations)

    # Fitting model to data
    model.fit(data)
    state_prob = model.predict_proba(data)
    pred_states = model.predict(data)

    if show == True:
        print('done')    

        # Print if model converged
        if conv == True:
            print('Converged: {}'.format(model.monitor_.converged))

    # Calculating predicted returns GIVEN current state
    # Transition matrix given the current state dot multiplied my the expected returns given state matrix
    pred_ret = np.dot(model.transmat_[pred_states[-1],:],model.means_)

    # Extracting current predicted state as calculated by the model
    pred_states = pred_states[-1]

    # Returning predicted return and predict states
    return(pred_ret,pred_states)

# Creating a function to convert predicted returns (% growth period-on-period) into predicted values
def pred_val(dprice,dpct):
    df = dprice
    name = df.columns[0]
    df.rename(columns ={df.columns[0]:'{} - actual'.format(name)},inplace = True)
    length = len(dpct.index)
    df = df.iloc[-length:]
    df = df.append(pd.DataFrame(index =(df.iloc[[-1]].index +pd.Timedelta(days=7))))
    df['{}'.format(name)] = dpct
    df['{}'.format(name)] = df['{} - actual'.format(name)] * (1+df['{}'.format(name)])
    df['{}'.format(name)] = df['{}'.format(name)].shift(1)
    df.drop('{} - actual'.format(name), axis=1, inplace=True)
    df = df.dropna()

    return(df)

prior_ret = pd.DataFrame() 
df_pred_states = pd.DataFrame() 
HMM_pred_val = pd.DataFrame() 
# TModel = False
# StatesModel = 2


MacroCheck = widgets.Checkbox(value = True,
                             description = "use macro?",
                             disabled = False,
                             indent = False)

StatesSlider = widgets.IntSlider(value=3,
                                min= 2,
                                max = 5,
                                step = 1,
                                description = "# of states",
                                orientation='horizontal',
                                readout=True,
                                readout_format='d')

TPeriod = widgets.IntText(value=520,
                         description ='Train window',
                         disabled=False)

HMMButton = widgets.Button(description='Run HMM', button_style='success')
HMMInputsBox = widgets.HBox([TPeriod, StatesSlider, MacroCheck, HMMButton])

HMMOutputBox = widgets.VBox([])
HMMBox = widgets.VBox([HMMInputsBox,HMMOutputBox])


def get_HMM(btn):
    global df_pct_ret
    global df_pct_vol
    global df_price
    global df_macro
    global prior_ret
    global df_pred_states
    global HMM_pred_val
    global TModel
    global StatesModel

    T = TPeriod.value
    states = StatesSlider.value
    macro = MacroCheck.value

    # Creating an empty dataframe to store predicted returns (priors for the Black Litterman Model)
    prior_ret = pd.DataFrame(index=df_pct_ret.dropna().index, columns = df_price.columns)
    prior_ret = prior_ret.iloc[T-1:]

    # Creating an empty dataframe to store predicted states
    df_pred_states = pd.DataFrame(index=df_pct_ret.dropna().index, columns = df_price.columns)
    df_pred_states = df_pred_states.iloc[T-1:]

    # Creating empty arrays for each Ticker to store their respective predicted returns (i.e. prior returns for Black Litterman)
    d = {name: pd.DataFrame(index=[df_pct_ret.dropna()],columns = df_price.columns) for name in df_price.columns}

    # Main calculation loop - For each ticker, train the model given user inputs (i.e. sliding window, states, macro, data) and predict returns
    for name in df_price.columns:
        time_walk = 0

        if macro == True:
            d[name] = pd.concat([df_pct_ret.loc[:,['{}'.format(name)]],df_pct_vol.loc[:,['{}'.format(name)]],df_macro], axis=1).dropna()

        else:
             d[name] = pd.concat([df_pct_ret.loc[:,['{}'.format(name)]],df_pct_vol.loc[:,['{}'.format(name)]]], axis=1).dropna()    

        for q in range(0,len(prior_ret.index)):

    #         print()
    #         print('{}: LOOP {} / {}'.format(name, (q+1),len(prior_ret.index)))

            pred_ret,pred_states = hmm_model(data=d[name].iloc[time_walk:time_walk+T],states=states)

            prior_ret[name].iloc[q:q+1] = pred_ret[0:1]
            df_pred_states[name].iloc[q:1+1] = pred_states

            time_walk +=1

    # print("Finished")   

    # Creating an HMM predicted value array
    HMM_pred_val = prior_ret.copy()
    HMM_pred_val.drop(HMM_pred_val.columns, axis=1, inplace=True)
    for q in range(0,len(prior_ret.columns)):
        new_pred_val = pred_val(df_price.iloc[:,q:q+1], prior_ret.iloc[:,q:q+1])
        HMM_pred_val = pd.concat([HMM_pred_val,new_pred_val], axis =1).dropna()

    HMMOutput = widgets.HTML(value='<b> DONE! </b>')
    HMMOutputBox.children = [HMMOutput]
    HMMOutputBox.layout.width = '100px'
    HideTemp2.children = [MainBoxTemp2]

HMMButton.on_click(get_HMM)

########################################################

ConfidenceInput = widgets.FloatSlider(value=0.95,
                                description = 'Confidence',
                                min =0,
                                max=1,
                                step= 0.01)

# TauInput = widgets.FloatSlider(value=0.05,
#                                description = 'Tau',
#                                min =0,
#                                max=1,
#                                step= 0.05)

RiskAversionInput = widgets.FloatSlider(value=1.0,
                              description = "Risk aversion",
                              min =0,
                              max=5,
                              step= 0.2)

TCovInput = widgets.IntText(value=260,
                         description ='Cov window',
                         disabled=False)

RfInput = widgets.FloatText(value=0.01,
                         description ='Risk-free',
                         disabled=False)

TCovInput.layout.width= "150px"
RfInput.layout.width = '150px'

BLButton = widgets.Button(description = "Run BL", button_style ="success")

BLInputBox = widgets.HBox([ConfidenceInput, RiskAversionInput, TCovInput, RfInput, BLButton])
# BLInputBox = widgets.HBox([ConfidenceInput, TauInput, RiskAversionInput, TCovInput, RfInput, BLButton])

BLHBox1 = widgets.HBox([])
BLHBox2 = widgets.HBox ([])
BLHBox3 = widgets.HBox([])

BLBox = widgets.VBox([BLInputBox,BLHBox1,BLHBox2,BLHBox3])


posterior_ret = pd.DataFrame()
BL_pred_val = pd.DataFrame()

def get_BL(btn):
    global prior_ret
    global df_mktcap
    global df_price
    confidences = ConfidenceInput.value
    global df_pct_ret
    global posterior_ret
    TCov = TCovInput.value
#     tau = TauInput.value
    risk_aversion = RiskAversionInput.value
    rf = RfInput.value

    K = min(len(prior_ret),len(df_mktcap))
    N = len(prior_ret.columns)
    BL_mcaps = df_mktcap.iloc[-K:]
    BL_prior = prior_ret.iloc[-K:]
    view_confidences = pd.Series(confidences, index =[df_price]) 

    # Creating an empty data frame to store Black Litterman posterior returns
    posterior_ret = pd.DataFrame(index=df_pct_ret.dropna().index, columns = df_price.columns)
    posterior_ret = posterior_ret.iloc[-K:]

    # Creating a covariance matrix for Black Litterman using Ledoit-Wolf covariance shrinkage method
    L = len(df_pct_ret)-K
    for Q in range(0,K):
        BL_cov_matrix = risk_models.CovarianceShrinkage(df_pct_ret.iloc[(L+Q+1-TCov):(L+Q+1)],returns_data=True,frequency=1).ledoit_wolf()
        bl = black_litterman.BlackLittermanModel(BL_cov_matrix, pi="market", absolute_views=pd.Series(BL_prior.iloc[Q]),omega="idzorek",view_confidences=view_confidences, market_caps = BL_mcaps.iloc[Q], risk_aversion = risk_aversion, risk_free_rate = rf)
        posterior_ret.iloc[Q] = bl.bl_returns()

    # Creating a BL predicted value array
    BL_pred_val = posterior_ret.copy()
    BL_pred_val.drop(BL_pred_val.columns, axis=1, inplace=True)
    for Q in range(0,len(posterior_ret.columns)):
        new_pred_val = pred_val(df_price.iloc[:,Q:Q+1], posterior_ret.iloc[:,Q:Q+1])
        BL_pred_val = pd.concat([BL_pred_val,new_pred_val], axis =1).dropna()

    # Plotting predicted data
    df_plot_val = pd.DataFrame(index=df_pct_ret.dropna().index)

    for name in df_price.columns:
        df_plot_val = pd.concat([df_plot_val,HMM_pred_val[name],BL_pred_val[name],df_price[name]],axis=1).dropna()
        df_plot_val.columns.values[-3] ='HMM - {}'.format(name)
        df_plot_val.columns.values[-2] ='BL - {}'.format(name)
        df_plot_val.columns.values[-1] ='Actual - {}'.format(name)


    for Q in range(0,len(df_plot_val.columns)):
        df_plot_val.iloc[:,Q] = pd.to_numeric(df_plot_val.iloc[:,Q], downcast="float")

    Plot_pred = bqv.InteractiveLinePlot(df_plot_val,title="HMM + BL predictions vs Actual")
    BLHBox1.children = [Plot_pred.show()]

    BL_aa = pd.DataFrame(index=posterior_ret.dropna().index, columns = df_price.columns)
    P = len(df_pct_ret)-len(posterior_ret)

    for Q in range(0,len(posterior_ret)):
        mu = posterior_ret.iloc[Q]*52
        S = risk_models.CovarianceShrinkage(df_pct_ret.iloc[(P+Q+1-TCov):(P+Q+1)],returns_data=True,frequency=52).ledoit_wolf()
        ef = efficient_frontier.EfficientFrontier(mu,S)
        ef.max_sharpe()
        weights = ef.clean_weights()
        df_weights = pd.DataFrame(weights, index=[0])
        BL_aa.iloc[Q] = df_weights.values

    for Q in range(0,len(BL_aa.columns)):
        BL_aa.iloc[:,Q] = pd.to_numeric(BL_aa.iloc[:,Q], downcast="float")


    Plot_aa = bqv.InteractiveLinePlot(BL_aa, title='BL Max Sharpe Asset Allocation')
    BLHBox2.children = [Plot_aa.show()]
    # ex2 = bqv.InteractiveLinePlot(BL_aa)
    # ex2.show()

    df_comp1t = pd.DataFrame(index=BL_aa.shift(1).dropna().index, columns = ["BL portfolio"])
    df_comp1 = pd.DataFrame(index=BL_aa.dropna().index, columns = ["BL portfolio"])
    BL_df_pct_ret = df_pct_ret.iloc[-len(df_comp1):]

    for Q in range(0,len(df_comp1t)):
        ret = np.dot(BL_aa.iloc[Q+1].T,BL_df_pct_ret.iloc[Q])
        df_comp1t.iloc[Q] = ret

    df_comp1.iloc[0]=100
    for Q in range(0,len(df_comp1t)):
        ret = df_comp1.iloc[Q]*(1+df_comp1t.iloc[Q])
        df_comp1.iloc[Q+1] = ret

    df_comp1 = pd.concat((df_comp1,df_benchmark),axis=1).dropna()
    df_comp1 = df_comp1.div(df_comp1.iloc[0]) *100
    df_comp1["diff"] = df_comp1.iloc[:,0] - df_comp1.iloc[:,1]

    for Q in range(0,len(df_comp1.columns)):
        df_comp1.iloc[:,Q] = pd.to_numeric(df_comp1.iloc[:,Q], downcast="float")

    Plot_perf = bqv.InteractiveLinePlot(df_comp1, title='BL Portfolio vs Benchmark')
    BLHBox3.children = [Plot_perf.show()]
    HideSaveBox.children =[SaveBox]

    # ex3 = bqv.InteractiveLinePlot(df_comp1)
    # ex3.show()

BLButton.on_click(get_BL)

SaveButton = widgets.Button(description = 'Save files for Part II', button_style ="warning")
SaveButton.layout.width = '300px'
SaveButton_label = widgets.Label('<-- CLICK HERE AND PROCEED TO SMARTER BETAS PART II (ref: GitHub link above)')
def get_Files(btn):
    # save to excel for convex optimizer
    writer = pd.ExcelWriter('BQuantResults.xlsx')
    posterior_ret.to_excel(writer, sheet_name='PostRet')
    df_pct_ret.to_excel(writer, sheet_name='PctChg')
    df_benchmark.to_excel(writer, sheet_name='Bench')

    writer.save()
    SaveButton_label.value = "Files Saved!"

SaveButton.on_click(get_Files)
SaveBox = widgets.HBox([SaveButton, SaveButton_label])

HideSaveBox = widgets.HBox([])
MainBoxTemp2 = widgets.VBox([BLBox, HideSaveBox])

HideTemp2 = widgets.VBox([])
MainBoxTemp1 = widgets.VBox([HMMBox, HideTemp2])

HideTemp1 = widgets.VBox([])
MainBox = widgets.VBox([BloombergBox,HideTemp1])
MainBox